## Preparing data for the database

Database tables:
 - suburbs [x]
 - routes [x]
 - terminals [x]
 - route_terminals [x]
 - route_agencies [x]
 - bus_stops [x]
 - vehicles [x]
 - routed_vehicles
 - users
 - trips
 - ml_model [x]
 - agencies [x]
 - agency_terminals [x]

In [192]:
import numpy as np
import pandas as pd

import geopandas as gpd
import matplotlib.pyplot as plt

# silence warnings
import warnings
warnings.filterwarnings("ignore")

In [193]:
# source files for GTFS data
src_agency = "../data_src/agency.txt"
src_calender = "../data_src/calendar.txt"
src_fare_attributes = "../data_src/fare_attributes.txt"
src_fare_rules = "../data_src/fare_rules.txt"
src_routes = "../data_src/routes.txt"
src_stops = "../data_src/stops.txt"
src_stop_times = "../data_src/stop_times.txt"
src_trips = "../data_src/trips.txt"
src_shapes = "../data_src/shapes.txt"

# destination files
dest_suburbs = "../data_src/to_db/suburbs.csv"
dest_terminals_suburb = "../data_src/to_db/terminals_with_suburb.csv"
dest_agencies = "../data_src/to_db/agencies.csv"
dest_terminals = "../data_src/to_db/terminals.csv"
dest_route_stops = "../data_src/to_db/route_stops.csv"
dest_agency_terminals = "../data_src/to_db/agency_terminals.csv"
dest_routes = "../data_src/to_db/routes.csv"
dest_route_terminals = "../data_src/to_db/route_terminals.csv"
dest_route_agency = "../data_src/to_db/route_agency.csv"
dest_bus_stops = "../data_src/to_db/bus_stops.csv"
dest_vehicles = "../data_src/to_db/vehicles.csv"
# dest_routed_vehicles = "../data_src/to_db/routed_vehicles.csv"
# dest_users = "../data_src/to_db/users.csv"
# dest_trips = "../data_src/to_db/trips.csv"
dest_ml_model = "../data_src/to_db/ml_model.csv"


# Load GTFS data
ag = pd.read_csv(src_agency, encoding='utf-8')
cal = pd.read_csv(src_calender, encoding='utf-8')
fare_attributes = pd.read_csv(src_fare_attributes, encoding='utf-8')
fare_rules = pd.read_csv(src_fare_rules, encoding='utf-8')
routes = pd.read_csv(src_routes, encoding='utf-8')
stops = pd.read_csv(src_stops, encoding='utf-8')
stops = pd.read_csv(src_stops, encoding='utf-8')
stop_times = pd.read_csv(src_stop_times, encoding='utf-8')
trips = pd.read_csv(src_trips, encoding='utf-8')
shapes = pd.read_csv(src_shapes, encoding='utf-8')


## Terminals, Bus stops and Suburbs


In [194]:

# All suburbs in Accra with their coordinates
suburb = {
    "Victoriaborg": [5.550725288945736, -0.19725829403485684], # [(community_name, lat, lon), ...]
    "East Ridge": [34.9949354690234, -85.22464835786998],
    "West Ridge": [5.558795615698128, -0.20771064828737007],
    "North Ridge":[5.567911999401812, -0.19222068441759996],
    "Usshertown": [5.543296982193049, -0.21309366240060734],
    "Osu": [5.557028446588415, -0.17581863016506272],
    "Mataheko": [5.566172156278807, -0.2481048116010064],
    "Cantonments": [5.576857274745243, -0.17830786879766952],
    "Labone":[5.568053292436006, -0.17267530114961677],
    "Airport Residential Area": [5.607572939497941, -0.18359947280974925],
    "Roman Ridge": [5.603481993918204, -0.1951557943127527],
    "Adabraka": [5.562988041560428, -0.21163055139305326],
    "Asylum Down": [5.568240727810714, -0.2071449749618672],
    "McCarthy Hill": [5.564390525232375, -0.29768226649148605],
    "Airport Hills": [5.615355794538434, -0.14341233294535413],
    "Dansoman": [5.54919153428196, -0.2561442383920633],
    "Adenta": [5.718137030616694, -0.15996542687023765],
    "Kaneshie": [5.576204124221918, -0.2446399153792841],
    "Kokomlemle": [5.5753228299629045, -0.20727450170570838],
    "West Legon/Westlands": [5.653988955662726, -0.20867821805619302],
    "Abelemkpe": [5.609534973765628, -0.21622119292250497],
    "Dzorwulu": [5.613794491770911, -0.19518629975927343],
    "East Legon": [5.6360822045306795, -0.1600751263095722],
    "Awoshie": [5.586535111369994, -0.2787234736062375],
    "Alajo": [5.598601443486721, -0.21726703878961862],
    "Kotobabi": [5.599329247147, -0.2034765971750303],
    "Christian Village": [5.631669983874022, -0.22324426057045038],
    "New Acra Town": [5.586988560337091, -0.21186187742239415],
    "Lartebiokorshie": [5.55113560682245, -0.23977708922626997],
    "Odorkor": [5.5760005505601296, -0.26211345003745906],
    "Labadi": [5.564830785424492, -0.15678855818819848],
    "Tesano": [5.605332278244894, -0.232951342562616],
    "Teshie": [5.5846148048627935, -0.10162353441753587],
    "Maamobi": [5.593574679422424, -0.1961926434785971],
    "Abeka": [5.595500717345612, -0.2426932071522069],
    "Lapaz": [5.610997264394199, -0.2485537718560147],
    "Nima": [5.585125208002253, -0.20073650235886942],
    "Madina": [5.67317139472009, -0.16565542452901147],
    "Achimota": [5.613677710598501, -0.23443089623190605],
    "Tema": [5.701213165254079, 0.021655983871629373],
    "Accra Metropolitan": [5.546496048340753, -0.2110071650158212],
    "Kwashieman": [5.593873593772809, -0.26950027220747463],
    "Darkuman": [5.5905625100726315, -0.250443279868646],
    "Akweteyman": [5.6130665369029025, -0.24023131812510604],
    "Abossey Okai":[5.5618019881333485, -0.23871373811404112],
    "Bubiashie":[5.57880787204536, -0.25019019298611717],
    "Mpoase": [5.528965632745328, -0.26469210623641665],
    "Sabon Zongo": [5.552473549406707, -0.2351653892896936],
    "Mamprobi": [5.536776738680185, -0.242251720725645],
    "Korle Gonno": [5.532862330505247, -0.22716195343750922],
    "Kanda": [5.576293150975519, -0.1935046064128002],
    "Chorkor": [5.5292220775596475, -0.24494512046607544],
    "Ashaiman": [5.693212320672842, -0.03347731311866802]
}

subs = {}
for suburb_name, coords in suburb.items():
    subs[suburb_name] = coords

suburbs_df = pd.DataFrame(subs, index=['latitude', 'longitude']).T.reset_index().rename(columns={'index': 'name'})
suburbs_df["suburb_id"] = suburbs_df.index + 1  # Create a unique ID for each suburb    


# Groupby stops and count the number of stop_times for each stop

stop_counts = stop_times.groupby('stop_id').size().reset_index(name='count')
stop_counts
# Merge the stop_counts with the stops DataFrame
stops_with_counts = pd.merge(stops, stop_counts, on='stop_id', how='left')\
                    .sort_values(by='count', ascending=False).reset_index(drop=True)

route_stops = stops_with_counts[stops_with_counts['stop_id'].apply(lambda x: x[0] != 'T')].reset_index(drop=True)

terminals = stops_with_counts[stops_with_counts['stop_id'].apply(lambda x: x[0] == 'T')].reset_index(drop=True)
bus_stops = stops_with_counts[stops_with_counts['stop_id'].apply(lambda x: x[0] == 'S')].reset_index(drop=True)
bus_stops["suburb_id"] = None

terminals.rename(columns={'stop_name': 'name', 'stop_lat':'latitude', 'stop_lon': 'longitude'}, inplace=True)
bus_stops.rename(columns={'stop_name': 'name', 'stop_lat':'latitude', 'stop_lon': 'longitude'}, inplace=True)

terminal_db_columns = ['stop_id', 'name', 'latitude', 'longitude', 'suburb_id']
bus_stops_db_columns = ['stop_id', 'name', 'latitude', 'longitude', 'suburb_id']

# for terminal suburb
term_sub = {
    'Terminal Afropa': 'Ablekuma North',
    'Terminal Abeka lapaz': 'lapaz',
    'Terminal Kaneshie Mkt Cmplx': 'Kaneshie',
    'Terminal Kaneshie Mkt': 'Kaneshie',
    'Terminal Nima Overhead Station': 'Nima',
    'Terminal Achimota Station': 'Achimota',
    'Terminal Accra New Tema Station': 'Accra Metropolitan',
    'Terminal Accra CMB': 'Usshertown',
    'Terminal Circle': 'Adabraka',
    'Terminal Achimota': 'Achimota',
    'Terminal Circle Odorna Station': 'Adabraka',
    'Terminal Dansoman Laststop': 'Mpoase',
    'Terminal Kwashieman Station': 'Kwashieman',
    'Terminal Korle Bu Station': 'Korle Gonno',
    'Terminal Kotobabi Down Station': 'Kotobabi',
    'Terminal Madina Station': 'Madina',
    'Terminal Odorkor Station': 'Odorkor',
    'Terminal Kasoa Station': 'Accra Metropolitan',
    'Terminal Osu Blow Up': 'Osu',
    'Terminal New Town Station': 'Accra Metropolitan',
    'Terminal Agbogbloshie Station No. 2': 'Usshertown',
    'Terminal Rawlings Park': 'Usshertown',
    'Terminal Zamrara Line': 'Dansoman',
    'Terminal Ashaiman Station': 'Ashaiman',
    'Terminal Arena Station': 'Accra Metropolitan'
}

# The rest let's put "---"
for terminal in terminals['name'].unique():
    if terminal not in term_sub:
        term_sub[terminal] = '---'

# add suburb_id to terminals
for terminal_name, suburb_name in term_sub.items():
    try:
        suburb_id = suburbs_df.loc[suburbs_df['name'] == suburb_name, 'suburb_id'].values[0] if suburb_name != '---' else None
    except IndexError:
        suburb_id = None
    terminals.loc[terminals['name'] == terminal_name, 'suburb_id'] = suburb_id

terminals.to_csv(dest_terminals, index=False, columns=terminal_db_columns)
bus_stops.to_csv(dest_bus_stops, index=False, columns=bus_stops_db_columns)
suburbs_df.to_csv(dest_suburbs, index=False)

print("======================================================")
print(f" data saved successfully to: \n\t{dest_terminals}\n\t{dest_suburbs}")
print("======================================================")


 data saved successfully to: 
	../data_src/to_db/terminals.csv
	../data_src/to_db/suburbs.csv


In [195]:
bus_stops

,stop_id,name,latitude,longitude,count,suburb_id
0,S569,Dzorwulu Junction,5.607488,-0.207327,12,None
1,S914,Korle Bu,5.536127,-0.226771,12,None
2,S937,Korle Bu,5.536194,-0.226506,12,None
3,S415,Pokuase Junction,5.687735,-0.281729,11,None
4,S880,Nyamekye 2,5.602377,-0.258796,10,None
...,...,...,...,...,...,...
2330,S2189,UBA,5.611580,-0.198684,1,None
2331,S455,CPF,5.627490,-0.249401,1,None
2332,S313,American House,5.649180,-0.150011,1,None
2333,S1317,Roundabout 185,5.535920,-0.213750,1,None


## Agencies

In [196]:
# Agency data cleanup

ag.agency_name.fillna('G.P.R.T.U', inplace=True)
ag.rename(columns={'agency_name': 'name'}, inplace=True)
ag.drop(columns=['agency_timezone'], inplace=True, errors='ignore')

ag.to_csv(dest_agencies, index=False, columns=['agency_id', 'name', 'agency_url'])
print("======================================================")
print(f" data saved successfully to: \n\t{dest_agencies}")
print("======================================================")

 data saved successfully to: 
	../data_src/to_db/agencies.csv


## Routes

In [197]:

print("-------------routes------------")
print(routes.isna().sum())


-------------routes------------
route_id             0
route_short_name     0
agency_id            0
route_long_name     32
route_type           0
dtype: int64


In [198]:
# routes.rename(columns={'route_short_name': 'name', 'route_long_name': 'description'}, inplace=True)
routes.drop(columns=['route_short_name', 'route_type'], inplace=True, errors='ignore')
routes.rename(columns={'route_long_name': 'name'}, inplace=True)
routes['distance_km'] = 0.0

routes['name'].fillna(routes['route_id'], inplace=True)
routes.sort_values(by='route_id', inplace=True)
routes.reset_index(drop=True, inplace=True)


# Use stop_times df to calculate distances between terminals for the total distance of each route. Using Haversine formula

def haversine_distance(lat1, lon1, lat2, lon2, unit='km'):
    """
    Calculate the great-circle distance between two points on the Earth using the Haversine formula.
    
    Parameters:
        lat1 (float): Latitude of the first point in degrees.
        lon1 (float): Longitude of the first point in degrees.
        lat2 (float): Latitude of the second point in degrees.
        lon2 (float): Longitude of the second point in degrees.
        unit (str): Unit of distance ('km' for kilometers, 'm' for meters). Default is 'km'.
    
    Returns:
        float: Distance between the two points in the specified unit.
    """
    # Radius of the Earth
    R = 6371.0 if unit == 'km' else 6371000.0  # km or meters
    
    # Convert latitude and longitude from degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    # Differences in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Distance
    distance = R * c
    return distance

stop_times_T = stop_times[stop_times['stop_id'].apply(lambda x: x[0] == 'T')].reset_index(drop=True)
stop_times_T['distance_km'] = 0.0

for i, row in stop_times_T.iterrows():

    if i == 0:
        continue
    try:
        prev_T = stop_times_T.iloc[i-1]['stop_id']
        curr_T = stop_times_T.iloc[i]['stop_id']

  
        prev_seq = stop_times_T.iloc[i-1]['stop_sequence']
        curr_seq = stop_times_T.iloc[i]['stop_sequence']

        if curr_seq > prev_seq:
            curr_R = stop_times_T.iloc[i]['trip_id'].split('_')[0]
            
            prev_T_lat = terminals.loc[terminals['stop_id'] == prev_T, 'latitude'].values[0]
            prev_T_lon = terminals.loc[terminals['stop_id'] == prev_T, 'longitude'].values[0]
            curr_T_lat = terminals.loc[terminals['stop_id'] == curr_T, 'latitude'].values[0]
            curr_T_lon = terminals.loc[terminals['stop_id'] == curr_T, 'longitude'].values[0]

            distance = haversine_distance(prev_T_lat, prev_T_lon, curr_T_lat, curr_T_lon, unit='km')

            # update distance in routes
            # if curr_T in routes['route_id'].values:
            routes.loc[routes['route_id'] == curr_R, 'distance_km'] = distance     


    except IndexError:
            continue

# Fix omissions
omitted_route_id = routes[routes.distance_km == 0.0]['route_id'].values
for omitted in omitted_route_id:
    stop_times[stop_times['trip_id'].apply(lambda x: x[0:-2] == omitted)].index
    init_stop = stop_times[stop_times['trip_id'].apply(lambda x: x[0:-2] == omitted)]['stop_id'].values[0]
    fin_stop = stop_times[stop_times['trip_id'].apply(lambda x: x[0:-2] == omitted)]['stop_id'].values[-1]

    if init_stop[0] == 'S':
        lt_, lg_ = bus_stops[bus_stops['stop_id'] == init_stop][['latitude', 'longitude']].values[0]
    elif init_stop[0] == 'T':
        lt_, lg_ = terminals[terminals['stop_id'] == init_stop][['latitude', 'longitude']].values[0]
    if fin_stop[0] == 'S':
        lt, lg = bus_stops[bus_stops['stop_id'] == fin_stop][['latitude', 'longitude']].values[0]
    elif fin_stop[0] == 'T':
        lt, lg = terminals[terminals['stop_id'] == fin_stop][['latitude', 'longitude']].values[0]

    distance = haversine_distance(lt_, lg_, lt, lg, unit='km')

    # Fill the distance in the routes DataFrame
    routes.loc[routes['route_id'] == omitted, 'distance_km'] = distance 

# Save the updated routes DataFrame to CSV
routes.to_csv(dest_routes, index=False, columns=['route_id', 'name', 'distance_km'])
print("======================================================")
print(f" data saved successfully to: \n\t{dest_routes}")
print("======================================================")

 data saved successfully to: 
	../data_src/to_db/routes.csv


##  Route_terminals and Route_agencies


In [199]:
# Route and terminals, and Route and stops
route_T_S = {'T':{}, 'S':{}}

trip_stops = stop_times[['trip_id','stop_id']].apply(lambda x: x[0:-2])\
    .groupby('trip_id')['stop_id'].apply(list).reset_index(name='stop_ids')

route_T_df = trip_stops.copy()
route_S_df = trip_stops.copy()

route_T_df['stop_ids'] = trip_stops['stop_ids'].apply(lambda x: [stop for stop in x if stop[0] == 'T']).values
route_T_df['trip_id'] = trip_stops['trip_id'].apply(lambda x: x[0:-2])
route_T_df.rename(columns={'trip_id': 'route_id'}, inplace=True)

route_S_df['stop_ids'] = trip_stops['stop_ids'].apply(lambda x: [stop for stop in x if stop[0] == 'S'])
route_S_df['trip_id'] = trip_stops['trip_id'].apply(lambda x: x[0:-2])
route_S_df.rename(columns={'trip_id': 'route_id'}, inplace=True)

# save route_T_df and route_S_df to CSV
route_T_df.to_csv(dest_route_terminals, index=False, columns=['route_id', 'stop_ids'])
route_S_df.to_csv(dest_route_stops, index=False, columns=['route_id', 'stop_ids'])
print("======================================================")
print(f" data saved successfully to: \n\t{dest_route_terminals}")
print("======================================================")


# Route and agencies
routes[['route_id', 'agency_id']].to_csv(dest_route_agency, index=False, columns=['route_id', 'agency_id'])
print("======================================================")
print(f" data saved successfully to: \n\t{dest_route_agency}")
print("======================================================")

 data saved successfully to: 
	../data_src/to_db/route_terminals.csv
 data saved successfully to: 
	../data_src/to_db/route_agency.csv


## Agency_terminals

In [200]:
# join routes and route_T_df on route_id
route_ag = pd.merge(routes, route_T_df, on='route_id', how='left')
# drop rows where stop_ids is NaN
route_ag = route_ag[route_ag['stop_ids'].notna()].reset_index(drop=True)

agency_terminals = route_ag[['agency_id', 'stop_ids']]

agency_terminals.to_csv(dest_agency_terminals, index=False, columns=['agency_id', 'stop_ids'])
print("======================================================")
print(f" data saved successfully to: \n\t{dest_agency_terminals}")
print("======================================================")

 data saved successfully to: 
	../data_src/to_db/agency_terminals.csv


In [201]:
terminals

,stop_id,name,latitude,longitude,count,suburb_id
0,T0,Terminal Abeka lapaz,5.607405,-0.247032,112,None
1,T75,Terminal Kaneshie Mkt Cmplx,5.565072,-0.235921,66,18
2,T136,Terminal Nima Overhead Station,5.581370,-0.198538,47,37
3,T251,Terminal Achimota Station,5.620789,-0.225837,46,39
4,T244,Terminal Accra New Tema Station,5.548629,-0.201709,44,41
...,...,...,...,...,...,...
225,T1789,Terminal Accra CMB 2,5.560070,-0.232294,1,None
226,T2694,Terminal Old Melcom 2,5.587400,-0.226946,1,None
227,T5559,Terminal Accra UTC Taxi Rank,5.547050,-0.210449,1,None
228,T5685,Terminal F,5.642330,-0.210306,1,None


In [202]:
routes

,route_id,agency_id,name,distance_km
0,002A,87,37 Lorry Park to Osu Blow Up,3.966941
1,002B,87,Osu Blow Up to 37 Lorry Park,3.966941
2,003A,63,37 Lorry Park to Tse Addo,4.556766
3,003B,63,Tse Addo to 37 Lorry Park,4.556766
4,004A,2,Abeka lapaz to Ashaiman Station,25.343253
...,...,...,...,...
646,997B,96,Circle Opps. Vodafone to UTC Station,0.466260
647,998A,0,Circle Overhead (Maame Dokono) to Ashaiman Sta...,23.882652
648,998B,0,Ashaiman Station to Circle Overhead Station,24.319993
649,999A,66,Abeka lapaz to Sowutuom Station,4.360529


In [203]:
# join routes and route_T_df on route_id
route_ag = pd.merge(routes, route_T_df, on='route_id', how='left')
# drop rows where stop_ids is NaN
route_ag = route_ag[route_ag['stop_ids'].notna()].reset_index(drop=True)

agency_terminals = route_ag[['agency_id', 'stop_ids']]
agency_terminals = agency_terminals.sort_values(by='agency_id').groupby('agency_id').agg({'stop_ids': lambda x: list(x)}).reset_index()

agency_terminals['stop_ids'] = agency_terminals['stop_ids'].apply(lambda x: [t_ for t in x for tt in t for t_ in t])
agency_terminals.to_csv(dest_agency_terminals, index=False, columns=['agency_id', 'stop_ids'])
print("======================================================")
print(f" data saved successfully to: \n\t{dest_agency_terminals}")
print("======================================================")

 data saved successfully to: 
	../data_src/to_db/agency_terminals.csv


In [204]:


agency_terminals.to_csv(dest_agency_terminals, index=False, columns=['agency_id', 'stop_ids'])

In [205]:
agency_terminals

,agency_id,stop_ids
0,0,"[T2688, T113, T2688, T113, T3443, T2196, T3443..."
1,1,"[T1043, T0, T1043, T0, T20, T1043, T20, T1043]"
2,2,"[T0, T6170, T0, T6170, T7, T0, T7, T0, T5548, ..."
3,3,"[T798, T0, T798, T0, T0, T288, T0, T288, T244,..."
4,4,"[T21, T75, T21, T75, T5559, T5561, T5559, T556..."
...,...,...
85,97,"[T1416, T1450, T1416, T1450, T1450, T1043, T14..."
86,98,"[T768, T3099, T768, T3099, T3099, T768, T3099,..."
87,99,"[T89, T103, T89, T103, T4715, T54, T4715, T54,..."
88,100,"[T54, T103, T54, T103]"


In [206]:
trips

,route_id,service_id,trip_id,shape_id
0,018A,service,018A_1,1001
1,005A,service,005A_1,2001
2,005B,service,005B_1,3001
3,016A,service,016A_1,4001
4,016B,service,016B_1,5001
...,...,...,...,...
646,055B,service,055B_1,647001
647,092A,service,092A_1,648001
648,092B,service,092B_1,649001
649,994B,service,994B_1,650001


## Creating Vehicles

In [208]:
import random

def generate_vehicle_number(region_groups, group_weights):
    """
    Generate a random Ghanaian vehicle number (license plate) in the format:
    [Region Code] [4-digit number]-[2-digit year]
    e.g., GT 1234-21
    """
    # Select a region group based on weights
    selected_group = random.choices(list(region_groups.keys()), weights=group_weights, k=1)[0]
    # Select a random region code from the chosen group
    region_code = random.choice(region_groups[selected_group])
    number = random.randint(1000, 9999)
    year = random.randint(15, 25)  # Years 2015–2025
    return f"{region_code} {number}-{year:02d}"

def generate_trotro_data(num_records=800):
    """
    Generate random trotro data with vehicle_number and capacity.
    
    Parameters:
        num_records (int): Number of trotro records to generate.
    
    Returns:
        pandas.DataFrame: DataFrame with columns 'vehicle_number' and 'capacity'.
    """
    # Define region groups and their weights
    region_groups = {
        'Greater Accra': ['GB', 'GC', 'GE', 'GG', 'GH', 'GL', 'GM', 'GN', 'GT', 'GS', 'GW', 'GX', 'GY'],
        'Central': ['CR'],
        'Western': ['WR', 'WT'],
        'Ashanti': ['AC', 'AE', 'AK', 'AP', 'AS', 'AW'],
        'Others': ['BA', 'BR', 'BW', 'BT', 'EN', 'ER', 'ES', 'NR', 'UE', 'UW', 'VA', 'VD', 'VR']
    }
    group_weights = [0.50, 0.15, 0.10, 0.15, 0.10]  # 50% Accra, 15% Central, 10% Western, 15% Ashanti, 10% Others

    # Define capacity weights
    capacity_weights = {
        '12-14': 0.7,  # 70% chance for 12–14 passengers
        '10-11': 0.1,  # 10% chance for 10–11 passengers
        '15-20': 0.15,  # 15% chance for 15–20 passengers
        '21-33': 0.05   # 5% chance for 21–33 passengers
    }

    # Ensure unique vehicle numbers
    vehicle_numbers = set()
    while len(vehicle_numbers) < num_records:
        vehicle_numbers.add(generate_vehicle_number(region_groups, group_weights))
    
    # Generate capacities with weighted probabilities
    capacities = []
    for _ in range(num_records):
        range_choice = random.choices(
            list(capacity_weights.keys()),
            weights=list(capacity_weights.values()),
            k=1
        )[0]
        if range_choice == '12-14':
            capacity = random.randint(12, 14)
        elif range_choice == '10-11':
            capacity = random.randint(10, 11)
        elif range_choice == '15-20':
            capacity = random.randint(15, 20)
        else:  # 21-33
            capacity = random.randint(21, 33)
        capacities.append(capacity)

    # Create DataFrame
    data = {
        'vehicle_number': list(vehicle_numbers),
        'capacity': capacities
    }
    return pd.DataFrame(data)

# Generate 800 trotro records
trotro_df = generate_trotro_data(num_records=1000)
trotro_df['latitude'] = 0.0
trotro_df['longitude'] = 0.0
# Optional: Save to CSV
trotro_df.to_csv(dest_vehicles, index=False, columns=['vehicle_number', 'capacity', 'latitude', 'longitude'])
print("======================================================")
print(f" data saved successfully to: \n\t{dest_vehicles}")
print("======================================================")

 data saved successfully to: 
	../data_src/to_db/vehicles.csv


## ML Models

In [209]:
"""
    name VARCHAR NOT NULL,
    version VARCHAR NOT NULL,
    task VARCHAR NOT NULL,
    url VARCHAR NOT NULL,
    inference_endpoint VARCHAR,
    description VARCHAR,
    target VARCHAR,
    features VARCHAR,
"""
models = [
    {
        'name': 'route_demand_prediction',
        'version': '1.0',
        'task': 'regression',
        'url': 'http://example.com/route_demand_prediction',
        'inference_endpoint': 'http://example.com/trotro_capacity_prediction/infer',
        'description': 'Predicts the capacity of trotros based on various features.',
        'target': 'route_demand',
        'features': 'vehicle_number, latitude, longitude'
    },
    {
        'name': 'travel_time_prediction',
        'version': '1.0',
        'task': 'regression',
        'url': 'http://example.com/travel_time_prediction',
        'inference_endpoint': 'http://example.com/travel_time_prediction/infer',
        'description': 'Predicts the travel time between two bus stops based on historical data.',
        'target': 'travel_time',
        'features': 'start_stop_id, end_stop_id, time_of_day, day_of_week, weather_conditions'
    },
    {
        'name': 'bus_stops_prediction',
        'version': '1.0',
        'task': 'classification',
        'url': 'http://example.com/bus_stops_prediction',
        'inference_endpoint': 'http://example.com/bus_stops_prediction/infer',
        'description': 'Classifies bus stops based on their usage patterns and features.',
        'target': 'stop_type',
        'features': 'stop_id, stop_name, latitude, longitude, usage_count, suburb_id'
    }

]


ml_model = pd.DataFrame(models, columns=['name', 'version', 'task', 'url', 'inference_endpoint', 'description', 'target', 'features'])
ml_model.to_csv(dest_ml_model, index=False)

print("======================================================")
print(f" data saved successfully to: \n\t{dest_ml_model}")
print("======================================================")

 data saved successfully to: 
	../data_src/to_db/ml_model.csv


In [210]:
ml_model


,name,version,task,url,inference_endpoint,description,target,features
0,route_demand_prediction,1.0,regression,http://example.com/route_demand_prediction,http://example.com/trotro_capacity_prediction/...,Predicts the capacity of trotros based on vari...,route_demand,"vehicle_number, latitude, longitude"
1,travel_time_prediction,1.0,regression,http://example.com/travel_time_prediction,http://example.com/travel_time_prediction/infer,Predicts the travel time between two bus stops...,travel_time,"start_stop_id, end_stop_id, time_of_day, day_o..."
2,bus_stops_prediction,1.0,classification,http://example.com/bus_stops_prediction,http://example.com/bus_stops_prediction/infer,Classifies bus stops based on their usage patt...,stop_type,"stop_id, stop_name, latitude, longitude, usage..."


In [213]:
len({'e':6})

1